In [18]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
datos = pd.read_excel(r"C:\Users\dapen\OneDrive - Universidad de los andes\MIAD\Visualización y Storytelling\MIAD-visualizacion\Datos.xlsx",
                      sheet_name='Export')
# Obtener columna Fecha_produccion: Fecha_vencimiento  - Vida_util
datos['Vida_util_timedelta'] = pd.to_timedelta(datos['Vida_util'], unit='D')
datos['Fecha_produccion'] = datos['Fecha_vencimiento'] - datos['Vida_util_timedelta']

# Split por espacio de la columna Referencia para obtener Marca, Empaque, Volumen, Und_volumen, Retornable y Texto
datos[['Marca', 'Empaque', 'Volumen', 'Und_Volumen', 'Retornable','Texto']] = datos['Referencia'].str.split(expand=True)

# Cambiar Retornable por Si
datos['Retornable'] = datos['Retornable'].replace('Retornable', 'SI')

# Creación de columnas año_produccion y mes_produccion, a partir de Fecha_produccion
datos['año_produccion'] = datos['Fecha_produccion'].dt.year
datos['mes_produccion'] = datos['Fecha_produccion'].dt.month

# Creación de columnas año_encuesta y mes_encuesta, a partir de Fecha_encuesta
datos['año_encuesta'] = datos['Fecha_encuesta'].dt.year
datos['mes_encuesta'] = datos['Fecha_encuesta'].dt.month

# Eliminación de columnas innecesarias para la visualización
datos = datos.drop(columns=['Referencia', 'Establecimiento', 'Lote', 'Vida_util_timedelta', 'Texto', 'Und_Volumen', 'Fecha_vencimiento'])

# Cálculo de la columna frescura teniendo en cuenta sus respectivas condiciones
def calculo_frescura(row):
    # Condiciones Marca_1, Marca_2, Marca_3
    if row['Marca'] in ['Marca_1', 'Marca_2', 'Marca_3']:
        if row['Edad_producto'] < 90:
            return 'Ideal'
        elif 90 < row['Edad_producto'] <= 120:
            return 'Debe Mejorar'
        else:
            return 'Inaceptable'
    # Condiciones Marca_4, Marca_5, Marca_6, Marca_7
    elif row['Marca'] in ['Marca_4', 'Marca_5', 'Marca_6', 'Marca_7']:
        if row['Edad_producto'] < 60:
            return 'Ideal'
        elif 60 < row['Edad_producto'] <= 100:
            return 'Debe Mejorar'
        else:
            return 'Inaceptable'

# Aplicar la función para crear la nueva columna
datos['Frescura'] = datos.apply(calculo_frescura, axis=1)

# Crear columnas con límite "Debe Mejorar" e "Inaceptable" para cada Marca

datos['Lim_debe_mejorar'] = pd.NA
datos['Lim_inaceptable'] = pd.NA

datos.loc[datos['Marca'].isin(['Marca_1', 'Marca_2', 'Marca_3']), 'Lim_debe_mejorar'] = 90
datos.loc[datos['Marca'].isin(['Marca_1', 'Marca_2', 'Marca_3']), 'Lim_inaceptable'] = 120

datos.loc[datos['Marca'].isin(['Marca_4', 'Marca_5', 'Marca_6', 'Marca_7']), 'Lim_debe_mejorar'] = 60
datos.loc[datos['Marca'].isin(['Marca_4', 'Marca_5', 'Marca_6', 'Marca_7']), 'Lim_inaceptable'] = 100

In [10]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13268 entries, 0 to 13267
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Ciudad            13268 non-null  object        
 1   Canal             13268 non-null  object        
 2   Edad_producto     13268 non-null  int64         
 3   Vida_util         13268 non-null  int64         
 4   Fecha_encuesta    13268 non-null  datetime64[ns]
 5   Fecha_produccion  13268 non-null  datetime64[ns]
 6   Marca             13268 non-null  object        
 7   Empaque           13268 non-null  object        
 8   Volumen           13268 non-null  object        
 9   Retornable        13268 non-null  object        
 10  año_produccion    13268 non-null  int32         
 11  mes_produccion    13268 non-null  int32         
 12  año_encuesta      13268 non-null  int32         
 13  mes_encuesta      13268 non-null  int32         
 14  Frescura          1326

In [6]:
datos['Ideal_Indicador'] = (datos['Frescura'] == 'Ideal').astype(int)

# Luego, realizamos un groupby por 'Mes' y 'Marca', y calculamos el total y la suma de 'Ideal_Indicador'
resultado = datos.groupby(['mes_encuesta', 'Marca']).agg(
    Total_Filas=('Ideal_Indicador', 'count'),
    Total_Ideal=('Ideal_Indicador', 'sum')
)

# Finalmente, calculamos la proporción
resultado['Proporción_Ideal'] = round((resultado['Total_Ideal'] / resultado['Total_Filas'])*100,0)

resultado

Total_Filas  Total_Ideal  Proporción_Ideal
mes_encuesta Marca                                              
1            Marca_1          341          239              70.0
             Marca_2          108           45              42.0
             Marca_3           18           17              94.0
             Marca_4          327          160              49.0
             Marca_5           27            7              26.0
...                           ...          ...               ...
12           Marca_3           26            8              31.0
             Marca_4          376          151              40.0
             Marca_5           43            7              16.0
             Marca_6          106            7               7.0
             Marca_7           64            9              14.0

[84 rows x 3 columns]